In [261]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.pipeline import Pipeline

In [262]:
# dataSet 불러오기

feature = ['sepal_length', 'sepal_width', 'petal length', 'petal width', 'class']
iris = pd.read_csv("dataset/iris.csv", names=feature)
iris_v = iris.iloc[:,:4]
iris_v

,sepal_length,sepal_width,petal length,petal width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


# 1. K-MEANS를 이용하여 Accuracy 측정

In [263]:
# K_MEANS
from sklearn.cluster import KMeans

model = KMeans(n_clusters=3,init='k-means++')
model.fit(iris_v)
predict = pd.Series(model.predict(iris_v)) # predict로 iris['class']와 accuracy 측정
predict

0      0
1      0
2      0
3      0
4      0
      ..
145    1
146    2
147    1
148    1
149    2
Length: 150, dtype: int32

In [264]:
# K_MEANS crosstab -> 데이터를 재구조화 시키고 싶은데 하는 방법을 모르겠음
# 따라서 crosstab으로 나타낸 뒤 가장 많이 겹치는 쪽으로 매핑하는 방향으로

le = preprocessing.LabelEncoder() # 먼저 iris['class']를 형변환
target = pd.DataFrame(le.fit_transform(iris['class']))

def mapping_func(target, predict): # 가장 정확도가 높은 mapping function
    predict_dict={}
    ct = pd.crosstab(target, predict)
    for i in range(0,len(ct),1):
        predict_dict[i] = ct[i].idxmax()
    return predict_dict

In [265]:
# 가장 정확도가 높은 쪽으로 매핑

predict_dict = mapping_func(target[0], predict)
predict = predict.map(predict_dict).astype('category')
predict

0      0
1      0
2      0
3      0
4      0
      ..
145    2
146    1
147    2
148    2
149    1
Length: 150, dtype: category
Categories (3, int64): [0, 1, 2]

In [266]:
# K_MEANS Accuracy Measurement
from sklearn.metrics import accuracy_score

accuracy_score(target, predict) # K_MEANS의 정확도는 약 89.33%

0.8933333333333333

# 2. DBSCAN을 이용하여 Accuracy 측정

In [267]:
# DBSCAN
from sklearn.cluster import DBSCAN

# create model and prediction
model = DBSCAN(eps=0.32,min_samples=5) # 거리 0.32, 주변 밀집도 5으로 설정
predict = pd.DataFrame(model.fit_predict(result))
predict.columns=['predict']

# concatenate labels to df as a new column
r = pd.concat([result,predict],axis=1)
r

NameError: name 'result' is not defined